In [1]:
import pandas as pd
import requests
import rdflib
from rdflib import URIRef
from rdflib.namespace import RDF
import xml.etree.ElementTree as ET
import networkx as nx
import regex as re
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import ast

In [2]:
df = pd.read_csv('data/person_review_name.csv', sep='\s*,\s*', engine='python')
df

,Person_1,Review_18,name_86,Unnamed: 3
0,http://data.persee.fr/authority/59214#Person,http://data.persee.fr/doc/jds_0021-8103_1975_n...,Henri-Jean Martin,NaN
1,http://data.persee.fr/authority/196935#Person,http://data.persee.fr/doc/jds_0021-8103_1963_n...,Jean Tulard,NaN
2,http://data.persee.fr/authority/251946#Person,http://data.persee.fr/doc/jds_0021-8103_1946_n...,André Piganiol,NaN
3,http://data.persee.fr/authority/235857#Person,http://data.persee.fr/doc/jds_0021-8103_1945_n...,Alfred Merlin,NaN
4,http://data.persee.fr/authority/235857#Person,http://data.persee.fr/doc/jds_0021-8103_1950_n...,Alfred Merlin,NaN
...,...,...,...,...
2956,http://data.persee.fr/authority/174841#Person,http://data.persee.fr/doc/jds_0021-8103_1969_n...,Jean Longnon,NaN
2957,http://data.persee.fr/authority/196742#Person,http://data.persee.fr/doc/jds_0021-8103_1970_n...,Pierre de La Coste-Messelière,NaN
2958,http://data.persee.fr/authority/622508#Person,http://data.persee.fr/doc/jds_0021-8103_1977_n...,Jean Richard,NaN
2959,http://data.persee.fr/authority/261328#Person,http://data.persee.fr/doc/jds_0021-8103_1969_n...,Marcel Durliat,NaN


In [3]:
len(df)

2961

In [ ]:
# Fonction de récupération et conversion du fichier RDF vers un XML plus facilement exploitable
# Entrée : URL / Sortie : return(XML)
def rdf_to_xml(url):
    try:
        g = rdflib.Graph().parse(url)
        serial = g.serialize(format='pretty-xml')
        root = ET.fromstring(serial)
        return(root)
    except:
        print('Bad news.')

# Fonction qui retourne le nom de l'auteur à partir du champs de citation biliographiee et l'id de la revue de publication (= jds le plus souvent)
# Entrée : liste d'URL / Sortie : liste de noms, d'URL en cas d'erreur
def url_to_name(cites_urls):
    cites_names = []
    id_publishs = []
    for cite in cites_urls:
        url = cite
        root = rdf_to_xml(url)
        try:
            id_publish = (re.findall('(?<=persee.fr\/doc\/)[a-z]+(?=_[0-9]+)', url))[0]
            id_publishs.append(id_publish)

            bib_cit = root.find('.//{http://purl.org/dc/terms/}bibliographicCitation').text
            name = (re.findall('^[^.]+', bib_cit))[0]
            cites_names.append(name)
        except:
            cites_names.append(url)
            id_publishs.append(id_publish)
    return(cites_names, id_publishs)

In [ ]:
# net_meta = nx.Graph()
net_meta = nx.DiGraph()

nb_a_traiter = len(df.index)

for i in range(nb_a_traiter):
# for i in range(100):
    url_df = df.loc[i]['Review_18']
    root = rdf_to_xml(url_df)

    # Récupération de la citation bibliographique contenue dans le graphe RDF
    bib_cit = root.find('.//{http://purl.org/dc/terms/}bibliographicCitation').text
    
    # Extraction du nom de l'auteur issu du dataframe
    name_df = df.loc[i]['name_86']
    # Extraction du nom de l'auteur issu issu de la citation bibliographique contenue dans le graphe RDF
    name_rdf_bib = (re.findall('^[^.]+', bib_cit))[0]

    # # Ancien bloc de comparaison des deux noms récupérés plus haut
    # if fuzz.token_sort_ratio(name_df, name_rdf_bib) >= 90:
    #     print(f'{name_df} = {name_rdf_bib}')

    cites_url = []
    isCitedBy_url = []

    # Boucle pour trouver la citation bibliographique
    for elem in root.iter('{http://purl.org/spar/cito/}cites'):
        cite_url = list(elem.attrib.values())[0]
        if cite_url not in cites_url:
            if cite_url != url_df:
                cites_url.append(cite_url)
    if cites_url:
        cites_names = (url_to_name(cites_url))[0]
        id_publishs = (url_to_name(cites_url))[1]

    # for elem in root.iter('{http://purl.org/spar/cito/}isCitedBy'):
    #     cite_url = list(elem.attrib.values())[0]
    #     if cite_url not in isCitedBy_url:
    #         if cite_url != url_df:
    #             print(f'Ça marche pour {i}')
    #             isCitedBy_url.append(cite_url)
    # if isCitedBy_url:
    #     isCitedBy_names = url_to_name(isCitedBy_url)

    # Boucle de remplissage du graphe
    if name_rdf_bib not in net_meta:
        net_meta.add_node(name_rdf_bib, corpus=True, publishing='jds', nPaper=1) #id=url, bib_cit=bib_cit, nCitation=0)
    else:
        net_meta.nodes[name_rdf_bib]["nPaper"] += 1

    for name_or_url, id_publish in zip(cites_names, id_publishs):
        if name_or_url not in net_meta:
            net_meta.add_node(name_or_url, corpus=False, publishing=id_publish, nPaper=1)
        if net_meta.has_edge(name_rdf_bib, name_or_url):
            net_meta[name_rdf_bib][name_or_url]["weight"] += 1
        else:
            net_meta.add_edge(name_rdf_bib, name_or_url, weight=1)


In [ ]:
print(len(net_meta))
nx.draw(net_meta)
nx.write_gexf(net_meta, f'graph_meta_{nb_a_traiter}.gexf')